DR Guia passo a passo para construir um modelo de rede neural profunda com Keras para prever Preços do Airbnb em Nova York e implantação como API REST usando Flask

Este guia permitirá implantar um modelo de aprendizado de máquina começando do zero. Aqui estão as etapas você vai cobrir:

• Defina seu objetivo
• Carregar dados
• Exploração de dados
• Preparação de dados
• Construa e avalie seu modelo
• Salve o modelo
• Construir API REST
• Implantar em produção

### Definir objetivo/meta

Obviamente, você precisa saber por que precisa de um modelo de aprendizado de máquina (ML).Conhecer o objetivo fornece insights sobre:

• O ML é a abordagem certa?
• De quais dados eu preciso?
• Qual será a aparência de um “bom modelo”? Quais métricas posso usar?
• Como resolvo o problema agora? Quão precisa é a solução?
• Quanto custará manter este modelo em funcionamento?

Em nosso exemplo, estamos tentando prever o preço de listagem do Airbnb³² por noite em Nova York. Nosso objetivo é claro - dados alguns dados, queremos que nosso modelo preveja quanto custará alugar um determinado imóvel
por noite.

### Carregar dados

Os dados vêm do Airbnb Open Data e estão hospedados no Kaggle³³

Desde 2008, hóspedes e anfitriões usam o Airbnb para ampliar as possibilidades de viagem e apresentar uma maneira mais única e personalizada de experimentar o mundo. Este conjunto de dados descreve
a atividade de listagem e métricas em NYC, NY para 2019.

#### Configuração 

Começaremos com várias importações e definindo uma semente aleatória para reprodutibilidade:

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
import joblib

In [ ]:
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 16, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [ ]:
df = pd.read_csv('airbnb_data\AB_NYC_2019.csv')

In [ ]:
df

Como podemos entender do que se tratam nossos dados?

### Data exploration

Esta etapa é crucial. O objetivo é obter uma melhor compreensão dos dados. Você pode ficar tentado
para impulsionar o processo de modelagem, mas isso seria abaixo do ideal. Olhando para grandes quantidades de
Por exemplo, procurar padrões e visualizar distribuições construirá sua intuição sobre os dados.
Essa intuição será útil ao modelar, imputar dados ausentes e observar valores discrepantes.
Uma maneira fácil de começar é contar o número de linhas e colunas em seu conjunto de dados:

In [ ]:
df.shape

Temos 48.895 linhas e 16 colunas. Dados suficientes para fazer algo interessante. Vamos começar com a variável que estamos tentando prever o preço. Para traçar a distribuição, usaremos

In [ ]:
sns.distplot(df.price);

Temos uma distribuição altamente distorcida com alguns valores na faixa de 10.000 (você pode querer explore-os). Usaremos um truque - transformação de log:

In [ ]:
sns.distplot(np.log1p(df.price));

Isso se parece mais com uma distribuição normal. Acontece que isso pode ajudar seu modelo a aprender melhor o dados³⁴. Você terá que se lembrar de pré-processar os dados antes de treinar e prever. O tipo de quarto parece ser outro ponto interessante. Vamos dar uma olhada:

In [ ]:
sns.countplot(x='room_type', data=df);

A maioria das listagens oferece lugares inteiros ou quartos privados. E quanto à localização? Que bairro grupos estão mais representados?

In [ ]:
sns.countplot(x='neighbourhood_group', data=df);

Como esperado, Manhattan lidera. Obviamente, o Brooklyn também está muito bem representado. Você pode
obrigado Mos Def, Nas, Masta Ace e Fabolous por isso.
Outra característica interessante é o número de avaliações. Vamos dar uma olhada nisso:

In [ ]:
sns.distplot(df.number_of_reviews);

Este parece seguir uma lei de potência³⁵ (tem cauda gorda). Este parece seguir uma lei de potência³⁶ (é tem uma cauda gorda). Parece haver alguns valores discrepantes (à direita) que podem ser de interesse para investigação.

#### Encontrando Correlações

A análise de correlação pode fornecer dicas sobre quais recursos podem ter poder preditivo quando treinando seu modelo.

Lembre-se, correlação não implica causalidade³⁷

Calcular o coeficiente de correlação de Pearson³⁸ entre um par de recursos é fácil:

In [ ]:
corr_matrix = df.corr();

Vejamos a correlação do preço com os demais atributos:

In [ ]:
price_corr = corr_matrix['price']
price_corr.iloc[price_corr.abs().argsort()]

O coeficiente de correlação é definido no intervalo de -1 a 1. Um valor próximo de 0 significa que não há correlação. O valor 1 sugere uma correlação positiva perfeita (por exemplo, à medida que o preço do Bitcoin aumenta, seus sonhos de possuir mais também estão aumentando!). O valor de -1 sugere correlação negativa perfeita (por exemplo, um grande número de críticas negativas deve estar correlacionado com preços mais baixos).

A correlação em nosso conjunto de dados parece muito ruim. Felizmente, os recursos categóricos não estão incluídos aqui. Eles também podem ter algum poder preditivo! Como podemos usá-los?

### Prepare os dados

O objetivo aqui é transformar os dados em um formato adequado ao seu modelo. Existem vários coisas que você deseja fazer ao lidar com dados estruturados (pense em CSV, banco de dados):

- Lidar com dados ausentes
- Remova colunas desnecessárias
- Transformar quaisquer recursos categóricos em números/vetores
- Dimensionar recursos numéricos